<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/ESTCScrape_chromedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#source: chatGPT and stack overflow

In [2]:
!pip install selenium --quiet
!pip install pandas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [3]:
pip install --upgrade selenium chromedriver-autoinstaller --quiet

In [4]:
#to try to bypass the site protection
!pip install fake-useragent --quiet


In [10]:
#### ATTEMPT 3.1 - AFTER ADDING SCROLLING METHOD AND CHECKING SELECTORS

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_element):
    """
    Scrape data from a single record element.
    """
    record_url = record_element.get_attribute('href')
    driver.get(record_url)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
        )
        # Extract data from the record page
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Name"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        date = driver.find_element(By.XPATH, '//td[text()="Date"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Date': date,
            'Title': title,
            'Year': year
        }
    except NoSuchElementException as e:
        print(f"Element not found in record {record_url}: {e}")
        return None
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scroll down and load more elements
def scroll_down():
    """
    Scroll down the page to load more elements.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust this delay as necessary based on your observation of the page load time
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Function to scrape records for a given year
def scrape_year(year):
    """
    Scrape records for a given year.
    """
    records = []
    driver.get(base_url)

    print(f"Navigating to search URL: {base_url} Year search: {year}")

    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
        )

        # Click the "Year" button to open the dropdown
        print("Clicking the 'Year' button")
        year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
        year_button.click()

        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
        )

        # Input the start year
        print(f"Inputting the start year: {year}")
        start_year_input = driver.find_element(By.CSS_SELECTOR, ".ais-RangeInput-input--min")
        start_year_input.clear()
        start_year_input.send_keys(str(year))

        # Input the end year
        print(f"Inputting the end year: {year}")
        end_year_input = driver.find_element(By.CSS_SELECTOR, ".ais-RangeInput-input--max")
        end_year_input.clear()
        end_year_input.send_keys(str(year))

        # Click the "Go" button
        print("Clicking the 'Go' button")
        go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
        go_button.click()

        while True:
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.XPATH, "//button[text()='View Record']"))
            )

            # Scroll down to load all records
            scroll_down()

            # Find all record elements
            record_elements = driver.find_elements(By.XPATH, "//button[text()='View Record']")#(By.CSS_SELECTOR, "div > button[type='View Record']") #"button:contains('View Record')")
            if not record_elements:
                print(f"No record elements found on the page for year {year}.")
                break

            for record_element in record_elements:
                print(f"Scraping record: {record_element.get_attribute('href')}")
                record_data = scrape_record(record_element)
                if record_data:
                    records.append(record_data)

                # Go back to the search results page
                driver.back()

                # Scroll down again to find the next "View Record" button
                scroll_down()

            # Check if there is a next page
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                if 'disabled' in next_button.get_attribute('class'):
                    print("Next button is disabled. No more pages.")
                    break
                else:
                    print("Clicking next button to go to the next page.")
                    next_button.click()
                    time.sleep(2)
            except NoSuchElementException:
                print("Next button not found. Assuming no more pages.")
                break
    except TimeoutException:
        print("Timed out waiting for records to load. Moving to the next page.")

    return records

### MAIN SCRIPT ###
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records_1473_1474.csv', index=False)
df.to_excel('ESTC_records_1473_1474.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473
Inputting the end year: 1473
Clicking the 'Go' button
Scraping record: None


InvalidArgumentException: Message: invalid argument: 'url' must be a string
  (Session info: chrome-headless-shell=125.0.6422.141)
Stacktrace:
#0 0x55854016ae3a <unknown>
#1 0x55853fe5445c <unknown>
#2 0x55853fee2f93 <unknown>
#3 0x55853fec34b2 <unknown>
#4 0x55853fee22cc <unknown>
#5 0x55853fec3253 <unknown>
#6 0x55853fe931c7 <unknown>
#7 0x55853fe93b3e <unknown>
#8 0x55854013127b <unknown>
#9 0x558540135327 <unknown>
#10 0x55854011ddae <unknown>
#11 0x558540135df2 <unknown>
#12 0x55854010274f <unknown>
#13 0x55854015a128 <unknown>
#14 0x55854015a2fb <unknown>
#15 0x558540169f6c <unknown>
#16 0x7b70b065bac3 <unknown>


In [12]:
### 3.2 - 3.1 WITH ADDITIONAL TECHNIQUES FOR BYPASS SECURITY

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from fake_useragent import UserAgent

# Set up Selenium with Chrome WebDriver and Fake User-Agent
ua = UserAgent()
user_agent = ua.random

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument(f"user-agent={user_agent}")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    """
    Scrape data from a single record URL.
    """
    driver.get(record_url)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
        )
        # Extract data from the record page
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Name"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        date = driver.find_element(By.XPATH, '//td[text()="Date"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Date': date,
            'Title': title,
            'Year': year
        }
    except NoSuchElementException as e:
        print(f"Element not found in record {record_url}: {e}")
        return None
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scroll down and load more elements
def scroll_down():
    """
    Scroll down the page to load more elements.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust this delay as necessary based on your observation of the page load time
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Function to scrape records for a given year
def scrape_year(year):
    """
    Scrape records for a given year.
    """
    records = []
    driver.get(base_url)

    print(f"Navigating to search URL: {base_url} Year search: {year}")

    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
        )

        # Click the "Year" button to open the dropdown
        print("Clicking the 'Year' button")
        year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
        year_button.click()

        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
        )

        # Input the start year
        print(f"Inputting the start year: {year}")
        start_year_input = driver.find_element(By.CSS_SELECTOR, ".ais-RangeInput-input--min")
        start_year_input.clear()
        start_year_input.send_keys(str(year))

        # Input the end year
        print(f"Inputting the end year: {year}")
        end_year_input = driver.find_element(By.CSS_SELECTOR, ".ais-RangeInput-input--max")
        end_year_input.clear()
        end_year_input.send_keys(str(year))

        # Click the "Go" button
        print("Clicking the 'Go' button")
        go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
        go_button.click()

        while True:
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.XPATH, "//button[text()='View Record']"))
            )

            # Scroll down to load all records
            scroll_down()

            # Find all record elements
            record_elements = driver.find_elements(By.XPATH, "//button[text()='View Record']")
            if not record_elements:
                print(f"No record elements found on the page for year {year}.")
                break

            for record_element in record_elements:
                record_url = record_element.get_attribute('href')
                print(f"Scraping record: {record_url}")
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

                # Go back to the search results page
                driver.back()

                # Scroll down again to find the next "View Record" button
                scroll_down()

            # Check if there is a next page
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                if 'disabled' in next_button.get_attribute('class'):
                    print("Next button is disabled. No more pages.")
                    break
                else:
                    print("Clicking next button to go to the next page.")
                    next_button.click()
                    time.sleep(2)
            except NoSuchElementException:
                print("Next button not found. Assuming no more pages.")
                break
    except TimeoutException:
        print("Timed out waiting for records to load. Moving to the next page.")

    return records

### MAIN SCRIPT ###
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records_1473_1474.csv', index=False)
df.to_excel('ESTC_records_1473_1474.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473
Inputting the end year: 1473
Clicking the 'Go' button
Scraping record: None


InvalidArgumentException: Message: invalid argument: 'url' must be a string
  (Session info: chrome-headless-shell=125.0.6422.141)
Stacktrace:
#0 0x5aca793e8e3a <unknown>
#1 0x5aca790d245c <unknown>
#2 0x5aca79160f93 <unknown>
#3 0x5aca791414b2 <unknown>
#4 0x5aca791602cc <unknown>
#5 0x5aca79141253 <unknown>
#6 0x5aca791111c7 <unknown>
#7 0x5aca79111b3e <unknown>
#8 0x5aca793af27b <unknown>
#9 0x5aca793b3327 <unknown>
#10 0x5aca7939bdae <unknown>
#11 0x5aca793b3df2 <unknown>
#12 0x5aca7938074f <unknown>
#13 0x5aca793d8128 <unknown>
#14 0x5aca793d82fb <unknown>
#15 0x5aca793e7f6c <unknown>
#16 0x7f5fe8f51ac3 <unknown>


In [8]:


# ###### ATTTEMPT 3 (FAIL) (trying to identify where time out occurs)


# #### next step: implement scroll down and check if the HTML codes are correct

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_url):
#     """
#     Scrape data from a single record URL.
#     """
#     driver.get(record_url)
#     try:
#         WebDriverWait(driver, 20).until(
#             EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
#         )
#         # Extract data from the record page
#         alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
#         author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
#         city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
#         country_name = driver.find_element(By.XPATH, '//td[text()="Country Name"]/following-sibling::td').text
#         estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
#         date = driver.find_element(By.XPATH, '//td[text()="Date"]/following-sibling::td').text
#         MARC_ID = driver.find_element(By.XPATH, '//td[text()="MARC ID"]/following-sibling::td').text
#         language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
#         title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
#         subtitle = driver.find_element(By.XPATH, '//td[text()="Subtitle"]/following-sibling::td').text
#         year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text
#         year_P = driver.find_element(By.XPATH, '//td[text()="Year (P&amp;P auto extracted)</"]/following-sibling::td').text

#         return {
#             'Alt Title': alt_title,
#             'Author': author,
#             'City': city,
#             'Country Code': country_code,
#             'ESTC No': estc_no,
#             'Format': format,
#             'Imprint Original': imprint_original,
#             'Language Code': language_code,
#             'Pagination': pagination,
#             'Title': title,
#             'Year': year
#         }
#     except NoSuchElementException as e:
#         print(f"Element not found in record {record_url}: {e}")
#         return None
#     except Exception as e:
#         print(f"Error scraping record {record_url}: {e}")
#         return None

# # Function to scrape records for a given year
# def scrape_year(year):
#     """
#     Scrape records for a given year.
#     """
#     records = []
#     driver.get(base_url)

#     print(f"Navigating to search URL: {base_url} Year search: {year}")

#     try:
#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
#         )

#         # Click the "Year" button to open the dropdown
#         print("Clicking the 'Year' button")
#         year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button") #click Year buttom, selector checked
#         year_button.click()

#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
#         )

#         # Input the start year
#         print(f"Inputting the start year: {year}")
#         start_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--min") # selector in the html code: "ais-RangeInput-input ais-RangeInput-input--min".
#         start_year_input.clear()
#         start_year_input.send_keys(str(year))

#         # Input the end year
#         print(f"Inputting the end year: {year}")
#         end_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--max") # selector in the html code: "ais-RangeInput-input ais-RangeInput-input--max"
#         end_year_input.clear()
#         end_year_input.send_keys(str(year))

#         # Click the "Go" button
#         print("Clicking the 'Go' button")
#         go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit") # selector checked
#         go_button.click()

#         while True:
#             WebDriverWait(driver, 60).until(
#                 EC.presence_of_element_located((By.XPATH, "//button[text()='View Record']")) #  button: <span class="ais-Highlight"><span class="ais-Highlight-nonHighlighted">1473</span></span></p>
# # <button>View Record</button>
# # </article>
# # </li>

#             )
#             record_elements = driver.find_elements(By.CSS_SELECTOR, "div > button[type='View Record']")  # I adjusted according to the  example in stack overflow: submit_button = driver.find_element(By.CSS_SELECTOR, "div > button[type='submit']") https://stackoverflow.com/questions/65657539/how-to-located-selenium-element-by-css-selector

#             #record_elements = driver.find_elements(By.XPATH, "//button[text()='View Record']")
#             if not record_elements:
#                 print(f"No record elements found on the page for year {year}.")
#                 break

#             record_urls = [element.get_attribute('href') for element in record_elements]
#             print(f"Found {len(record_urls)} records on the page.")

#             for record_url in record_urls:
#                 print(f"Scraping record: {record_url}")
#                 record_data = scrape_record(record_url)
#                 if record_data:
#                     records.append(record_data)

#             # Check if there is a next page
#             try:
#                 next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#                 if 'disabled' in next_button.get_attribute('class'):
#                     print("Next button is disabled. No more pages.")
#                     break
#                 else:
#                     print("Clicking next button to go to the next page.")
#                     next_button.click()
#                     time.sleep(2)
#             except NoSuchElementException:
#                 print("Next button not found. Assuming no more pages.")
#                 break
#     except TimeoutException:
#         print("Timed out waiting for records to load. Moving to the next page.")

#     return records

# ### TEST 1473-1475 #####
# # Scrape records year by year
# all_records = []
# for year in range(1473, 1475):
#     print(f"Scraping year: {year}")
#     records = scrape_year(year)
#     all_records.extend(records)

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
# df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473
Inputting the end year: 1473
Clicking the 'Go' button
No record elements found on the page for year 1473.
Scraping year: 1474
Navigating to search URL: https://estc.printprobability.org/ Year search: 1474
Clicking the 'Year' button
Inputting the start year: 1474
Inputting the end year: 1474
Clicking the 'Go' button
No record elements found on the page for year 1474.


In [8]:
# ###### ATTTEMPT 3 (FAIL) (trying to identify where time out occurs)

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_url):
#     """
#     Scrape data from a single record URL.
#     """
#     driver.get(record_url)
#     try:
#         WebDriverWait(driver, 20).until(
#             EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
#         )
#         # Extract data from the record page
#         alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
#         author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
#         city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
#         country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
#         estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
#         format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
#         imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
#         language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
#         pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
#         title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
#         year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

#         return {
#             'Alt Title': alt_title,
#             'Author': author,
#             'City': city,
#             'Country Code': country_code,
#             'ESTC No': estc_no,
#             'Format': format,
#             'Imprint Original': imprint_original,
#             'Language Code': language_code,
#             'Pagination': pagination,
#             'Title': title,
#             'Year': year
#         }
#     except NoSuchElementException as e:
#         print(f"Element not found in record {record_url}: {e}")
#         return None
#     except Exception as e:
#         print(f"Error scraping record {record_url}: {e}")
#         return None

# # Function to scrape records for a given year
# def scrape_year(year):
#     """
#     Scrape records for a given year.
#     """
#     records = []
#     driver.get(base_url)

#     print(f"Navigating to search URL: {base_url} Year search: {year}")

#     try:
#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
#         )

#         # Click the "Year" button to open the dropdown
#         print("Clicking the 'Year' button")
#         year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
#         year_button.click()

#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
#         )

#         # Input the start year
#         print(f"Inputting the start year: {year}")
#         start_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--min")
#         start_year_input.clear()
#         start_year_input.send_keys(str(year))

#         # Input the end year
#         print(f"Inputting the end year: {year}")
#         end_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--max")
#         end_year_input.clear()
#         end_year_input.send_keys(str(year))

#         # Click the "Go" button
#         print("Clicking the 'Go' button")
#         go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
#         go_button.click()

#         while True:
#             WebDriverWait(driver, 60).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
#             )
#             record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")

#             if not record_elements:
#                 print(f"No record elements found on the page for year {year}.")
#                 break

#             record_urls = [element.get_attribute('href') for element in record_elements]
#             print(f"Found {len(record_urls)} records on the page.")

#             for record_url in record_urls:
#                 print(f"Scraping record: {record_url}")
#                 record_data = scrape_record(record_url)
#                 if record_data:
#                     records.append(record_data)

#             # Check if there is a next page
#             try:
#                 next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#                 if 'disabled' in next_button.get_attribute('class'):
#                     print("Next button is disabled. No more pages.")
#                     break
#                 else:
#                     print("Clicking next button to go to the next page.")
#                     next_button.click()
#                     time.sleep(2)
#             except NoSuchElementException:
#                 print("Next button not found. Assuming no more pages.")
#                 break
#     except TimeoutException:
#         print("Timed out waiting for records to load. Moving to the next page.")

#     return records

# ### TEST 1473-1475 #####
# # Scrape records year by year
# all_records = []
# for year in range(1473, 1475):
#     print(f"Scraping year: {year}")
#     records = scrape_year(year)
#     all_records.extend(records)

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
# df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# # Close the driver
# driver.quit()

Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome-headless-shell=125.0.6422.141)
Stacktrace:
#0 0x58005b4d5e3a <unknown>
#1 0x58005b1bf2a1 <unknown>
#2 0x58005b201a93 <unknown>
#3 0x58005b22e4b2 <unknown>
#4 0x58005b1ffa79 <unknown>
#5 0x58005b22e67e <unknown>
#6 0x58005b24d2cc <unknown>
#7 0x58005b22e253 <unknown>
#8 0x58005b1fe1c7 <unknown>
#9 0x58005b1feb3e <unknown>
#10 0x58005b49c27b <unknown>
#11 0x58005b4a0327 <unknown>
#12 0x58005b488dae <unknown>
#13 0x58005b4a0df2 <unknown>
#14 0x58005b46d74f <unknown>
#15 0x58005b4c5128 <unknown>
#16 0x58005b4c52fb <unknown>
#17 0x58005b4d4f6c <unknown>
#18 0x7e81fcd56ac3 <unknown>


In [7]:
# ###### ATTTEMPT 2 (FAIL) (trying to extract some records from the view record buttom - its not finding the records)

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_element):
#     """
#     Scrape data from a single record element.
#     """
#     try:
#         # Extract data from the record element
#         author = record_element.find_element(By.CSS_SELECTOR, 'div.authors').text.strip()
#         estc_no = record_element.find_element(By.CSS_SELECTOR, 'div.estcNo').text.strip()
#         year = record_element.find_element(By.CSS_SELECTOR, 'div.year').text.strip()

#         return {
#             'Author': author,
#             'ESTC No': estc_no,
#             'Year': year
#         }
#     except Exception as e:
#         print(f"Error scraping record: {e}")
#         return None

# # Function to scrape records from the current page
# def scrape_current_page():
#     """
#     Scrape records from the current page.
#     """
#     records = []
#     try:
#         record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record")
#         print(f"Found {len(record_elements)} records on the page.")

#         for record_element in record_elements:
#             record_data = scrape_record(record_element)
#             if record_data:
#                 records.append(record_data)

#         return records
#     except Exception as e:
#         print(f"Error scraping current page: {e}")
#         return None

# # Function to check if there's a next page and navigate to it
# def go_to_next_page():
#     """
#     Check if there's a next page and navigate to it.
#     """
#     try:
#         next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#         if 'disabled' not in next_button.get_attribute('class'):
#             print("Clicking next button to go to the next page.")
#             next_button.click()
#             time.sleep(2)
#             return True
#         else:
#             print("Next button is disabled. No more pages.")
#             return False
#     except NoSuchElementException:
#         print("Next button not found. Assuming no more pages.")
#         return False

# ### MAIN SCRIPT ###

# # Navigate to the base URL
# driver.get(base_url)

# # Scrape records from the main page and navigate to the next page if available
# all_records = []
# while True:
#     print("Scraping records from the current page.")
#     records = scrape_current_page()
#     if records:
#         all_records.extend(records)

#     if not go_to_next_page():
#         break

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records_main_page.csv', index=False)
# df.to_excel('ESTC_records_main_page.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping records from the current page.
Found 0 records on the page.
Next button not found. Assuming no more pages.


In [ ]:
# ### ATTEMPT 5 - TRYING TO BYPASS ESTC PROTECTION

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# import random
# from selenium.common.exceptions import TimeoutException, NoSuchElementException
# from fake_useragent import UserAgent

# # Generate a random user agent
# ua = UserAgent()
# user_agent = ua.random

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
# chrome_options.add_argument(f'user-agent={user_agent}')

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_element):
#     """
#     Scrape data from a single record element.
#     """
#     try:
#         # Extract data from the record element
#         author = record_element.find_element(By.CSS_SELECTOR, 'div.authors').text.strip()
#         estc_no = record_element.find_element(By.CSS_SELECTOR, 'div.estcNo').text.strip()
#         year = record_element.find_element(By.CSS_SELECTOR, 'div.year').text.strip()

#         return {
#             'Author': author,
#             'ESTC No': estc_no,
#             'Year': year
#         }
#     except Exception as e:
#         print(f"Error scraping record: {e}")
#         return None

# # Function to scrape records from the current page
# def scrape_current_page():
#     """
#     Scrape records from the current page.
#     """
#     records = []
#     try:
#         # Wait for the records to load
#         WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.record")))

#         record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record")
#         print(f"Found {len(record_elements)} records on the page.")

#         for record_element in record_elements:
#             record_data = scrape_record(record_element)
#             if record_data:
#                 records.append(record_data)

#         return records
#     except Exception as e:
#         print(f"Error scraping current page: {e}")
#         return None

# # Function to check if there's a next page and navigate to it
# def go_to_next_page():
#     """
#     Check if there's a next page and navigate to it.
#     """
#     try:
#         next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#         if 'disabled' not in next_button.get_attribute('class'):
#             print("Clicking next button to go to the next page.")
#             next_button.click()
#             time.sleep(random.uniform(2, 5))  # Random sleep to mimic human behavior
#             return True
#         else:
#             print("Next button is disabled. No more pages.")
#             return False
#     except NoSuchElementException:
#         print("Next button not found. Assuming no more pages.")
#         return False

# ### MAIN SCRIPT ###

# # Navigate to the base URL
# driver.get(base_url)

# # Scrape records from the main page and navigate to the next page if available
# all_records = []
# while True:
#     print("Scraping records from the current page.")
#     records = scrape_current_page()
#     if records:
#         all_records.extend(records)

#     if not go_to_next_page():
#         break

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records_main_page.csv', index=False)
# df.to_excel('ESTC_records_main_page.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping records from the current page.
Error scraping current page: Message: 

Next button not found. Assuming no more pages.


In [ ]:
# ## ATTEMPT 4 BYPASS

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# import random
# from selenium.common.exceptions import TimeoutException, NoSuchElementException
# from fake_useragent import UserAgent

# # Generate a random user agent
# ua = UserAgent()
# user_agent = ua.random

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
# chrome_options.add_argument(f'user-agent={user_agent}')

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_element):
#     """
#     Scrape data from a single record element.
#     """
#     try:
#         # Extract data from the record element
#         author = record_element.find_element(By.CSS_SELECTOR, 'div.authors').text.strip()
#         estc_no = record_element.find_element(By.CSS_SELECTOR, 'div.estcNo').text.strip()
#         year = record_element.find_element(By.CSS_SELECTOR, 'div.year').text.strip()

#         return {
#             'Author': author,
#             'ESTC No': estc_no,
#             'Year': year
#         }
#     except Exception as e:
#         print(f"Error scraping record: {e}")
#         return None

# # Function to scrape records from the current page
# def scrape_current_page():
#     """
#     Scrape records from the current page.
#     """
#     records = []
#     try:
#         # Wait for the records to load
#         WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.record")))

#         record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record")
#         print(f"Found {len(record_elements)} records on the page.")

#         for record_element in record_elements:
#             record_data = scrape_record(record_element)
#             if record_data:
#                 records.append(record_data)

#         return records
#     except Exception as e:
#         print(f"Error scraping current page: {e}")
#         return None

# # Function to check if there's a next page and navigate to it
# def go_to_next_page():
#     """
#     Check if there's a next page and navigate to it.
#     """
#     try:
#         next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#         if 'disabled' not in next_button.get_attribute('class'):
#             print("Clicking next button to go to the next page.")
#             next_button.click()
#             time.sleep(random.uniform(2, 5))  # Random sleep to mimic human behavior
#             return True
#         else:
#             print("Next button is disabled. No more pages.")
#             return False
#     except NoSuchElementException:
#         print("Next button not found. Assuming no more pages.")
#         return False

# ### MAIN SCRIPT ###

# # Navigate to the base URL
# driver.get(base_url)

# # Scrape records from the main page and navigate to the next page if available
# all_records = []
# while True:
#     print("Scraping records from the current page.")
#     records = scrape_current_page()
#     if records:
#         all_records.extend(records)

#     if not go_to_next_page():
#         break

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records_main_page.csv', index=False)
# df.to_excel('ESTC_records_main_page.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping records from the current page.
Error scraping current page: Message: 

Next button not found. Assuming no more pages.


In [ ]:
# ## ATEMPT 6 BYPASS

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# import random
# from selenium.common.exceptions import TimeoutException, NoSuchElementException
# from fake_useragent import UserAgent

# # Generate a random user agent
# ua = UserAgent()
# user_agent = ua.random

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
# chrome_options.add_argument(f'user-agent={user_agent}')

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_element):
#     """
#     Scrape data from a single record element.
#     """
#     try:
#         # Extract data from the record element
#         author = record_element.find_element(By.CSS_SELECTOR, 'div.authors').text.strip()
#         estc_no = record_element.find_element(By.CSS_SELECTOR, 'div.estcNo').text.strip()
#         year = record_element.find_element(By.CSS_SELECTOR, 'div.year').text.strip()

#         return {
#             'Author': author,
#             'ESTC No': estc_no,
#             'Year': year
#         }
#     except Exception as e:
#         print(f"Error scraping record: {e}")
#         return None

# # Function to scrape records from the current page
# def scrape_current_page():
#     """
#     Scrape records from the current page.
#     """
#     records = []
#     try:
#         # Wait for the records to load
#         WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.record")))

#         record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record")
#         print(f"Found {len(record_elements)} records on the page.")

#         for record_element in record_elements:
#             record_data = scrape_record(record_element)
#             if record_data:
#                 records.append(record_data)

#         return records
#     except Exception as e:
#         print(f"Error scraping current page: {e}")
#         return None

# # Function to check if there's a next page and navigate to it
# def go_to_next_page():
#     """
#     Check if there's a next page and navigate to it.
#     """
#     try:
#         next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#         if 'disabled' not in next_button.get_attribute('class'):
#             print("Clicking next button to go to the next page.")
#             next_button.click()
#             time.sleep(random.uniform(2, 5))  # Random sleep to mimic human behavior
#             return True
#         else:
#             print("Next button is disabled. No more pages.")
#             return False
#     except NoSuchElementException:
#         print("Next button not found. Assuming no more pages.")
#         return False

# ### MAIN SCRIPT ###

# # Navigate to the base URL
# driver.get(base_url)

# # Scrape records from the main page and navigate to the next page if available
# all_records = []
# while True:
#     print("Scraping records from the current page.")
#     records = scrape_current_page()
#     if records:
#         all_records.extend(records)

#     if not go_to_next_page():
#         break

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records_main_page.csv', index=False)
# df.to_excel('ESTC_records_main_page.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping records from the current page.
Error scraping current page: Message: 

Next button not found. Assuming no more pages.
